<a href="https://colab.research.google.com/github/visiont3lab/deep-learning-course/blob/main/colab/ClassificationSimple.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Library

* [Softmax](https://machinelearningmastery.com/softmax-activation-function-with-python/#:~:text=Softmax%20is%20a%20mathematical%20function,each%20value%20in%20the%20vector.&text=Each%20value%20in%20the%20output,of%20membership%20for%20each%20class.)
[math Softmax](https://ljvmiranda921.github.io/notebook/2017/08/13/softmax-and-the-negative-log-likelihood/)
* [Pytorch training](https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html)

In [4]:
from torch import nn
from torch.utils.data import DataLoader
from torch import optim
import torch
from torch import nn
from torchsummary import summary
#!pip install torchsummary
import torch.nn.functional as F
from torch.utils.data import TensorDataset,Dataset
from torchvision import datasets
from torchvision import transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import numpy as np
import plotly.graph_objects as go
# Loss function pytorch: https://neptune.ai/blog/pytorch-loss-functions
import copy
import pandas as pd
from datetime import datetime

## Load Data

In [5]:
#!pip install pandas

import pandas as pd
df = pd.read_csv("https://raw.githubusercontent.com/visiont3lab/deep-learning-course/main/data/covid19-ita-regioni.csv")
df.to_csv("kldskd.csv")

lx = ["terapia_intensiva","isolamento_domiciliare","totale_positivi","dimessi_guariti","deceduti","tamponi","indice_rt"]
ly = ["zona"]
X = df[lx].values
Y = df[ly].values

a = {
    "bianca": 0,
    "gialla": 1,
    "arancione": 2,
    "rossa": 3,
}

Y = []
for el in df["zona"].tolist():
  Y.append( a[el] )

Y = [a[el] for el in df["zona"].tolist()]
Y = np.array(Y)

In [6]:
set(df["zona"].tolist())
df["zona"].unique()

array(['arancione', 'rossa', 'gialla', 'bianca'], dtype=object)

## Dataset

In [7]:
from sklearn.model_selection import train_test_split
# Training and Test Set
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.3,shuffle=True,random_state=2)
print(f"X Train shape: {X_train.shape} , X Test shape: {X_test.shape}")

X Train shape: (1882, 7) , X Test shape: (807, 7)


In [8]:
from torch.utils.data import Dataset,DataLoader

class MyDatasets(Dataset):
  def __init__(self,x,y):
    self.x = torch.tensor(x,dtype=torch.float32)
    #self.x = (self.x - torch.mean(self.x) ) / torch.std(self.x)
    self.y = torch.tensor(y,dtype=torch.int64) 
  def __getitem__(self,index):
    return self.x[index], self.y[index]
  def __len__(self):
    #print(self.y.shape[0])
    return self.y.shape[0]

# Load training data
train_ds = MyDatasets(X_train,Y_train)
train_dl = DataLoader(train_ds,batch_size=10,shuffle=True)

# Load test data
test_ds = MyDatasets(X_test,Y_test)
test_dl = DataLoader(test_ds,batch_size=10,shuffle=True)

#ds.__getitem__(1)
#for x,y in train_dl:
#  print(x,y)
#  break

## Neural Network


In [9]:
'''
# Softmax
y = torch.log_softmax( torch.tensor( [1,6,3], dtype=torch.float32), dim=0 ) 
y1 = torch.softmax( torch.tensor( [1,6,3], dtype=torch.float32), dim=0 ) 

print(y)
print(y1)

p1 = np.log( np.exp(1)/ ( np.exp(1)+np.exp(2)+np.exp(3) ) )
p2 = np.log( np.exp(2)/ ( np.exp(1)+np.exp(2)+np.exp(3) ) )
p3 = np.log( np.exp(3)/ ( np.exp(1)+np.exp(2)+np.exp(3) ) )

print(np.round(p1,4),np.round(p2,4),np.round(p3,4))

torch.argmax( y )
torch.argmax( y1 )
'''

'\n# Softmax\ny = torch.log_softmax( torch.tensor( [1,6,3], dtype=torch.float32), dim=0 ) \ny1 = torch.softmax( torch.tensor( [1,6,3], dtype=torch.float32), dim=0 ) \n\nprint(y)\nprint(y1)\n\np1 = np.log( np.exp(1)/ ( np.exp(1)+np.exp(2)+np.exp(3) ) )\np2 = np.log( np.exp(2)/ ( np.exp(1)+np.exp(2)+np.exp(3) ) )\np3 = np.log( np.exp(3)/ ( np.exp(1)+np.exp(2)+np.exp(3) ) )\n\nprint(np.round(p1,4),np.round(p2,4),np.round(p3,4))\n\ntorch.argmax( y )\ntorch.argmax( y1 )\n'

In [10]:
from torch import nn

# rete 7 input , 100 neuroni (primo hidden layer), 50 neuroni (secondo hidden layer, 4 neuroni nell'ouput layer)
class NeuralNet(nn.Module):
  def __init__(self):
    super(NeuralNet,self).__init__()
    num_inputs = 7
    num_outputs = 4
    self.fc1 = nn.Linear(7,100) 
    self.fc2 = nn.Linear(100,50) 
    self.fc3 = nn.Linear(50,4) 
  def forward(self,x):
    x = torch.tanh( self.fc1(x) )
    x = torch.tanh( self.fc2(x) )
    x = torch.tanh( self.fc3(x) )
    x = torch.log_softmax( x, dim=-1 ) #NNLL
    return x

model = NeuralNet()

# Visualiza parameters
#for name, param in model.named_parameters():
#  print(name,param)

#inp = torch.tensor([5,5],dtype=torch.float32)
#y = model.forward(inp)
#print(y)
#torch.argmax(y)

# Test
fake_input = torch.randn((1,7))
y_hat = model(fake_input)
print(y_hat)

tensor([[-1.3026, -1.5162, -1.3723, -1.3660]], grad_fn=<LogSoftmaxBackward0>)


## Training

In [11]:
import torch

# Inizilizzo il modello
model = NeuralNet()

# Definiscoo la loss
criterion = torch.nn.NLLLoss(reduction="sum")

# Definisco l'optimizer
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

for epoch in range(100):  # loop over the dataset multiple times

    # Training su tutto il dataset
    running_loss = 0.0
    for i, data in enumerate(train_dl, 0):
       
        # prendo i dati (a gruppi batch size)
        inputs, labels = data
        #print(inputs, labels)
      
        # zero the parameter gradients (resetto i pesi )
        optimizer.zero_grad()

        # forward (trovo i valori stimati )
        outputs = model(inputs)

        # Mi calcolo l'errore
        loss = criterion(outputs, labels)

        # back propagation (trovare la derivata dell loss function)
        loss.backward()

        # aggiorna i pesi
        optimizer.step()

        # stampi la loss attuale
        running_loss += loss.item()

        # Alla fine di un aggiornamente --> Quanto è buono il mio modello?
        total_corrects = 0
        for xtest,ytest in test_dl:
          ytest_hat = torch.argmax( model(xtest) , dim=-1, keepdim=True )
          corrects = ytest_hat.eq(ytest.reshape(ytest_hat.shape)).sum().item()
          total_corrects = total_corrects + corrects

        print(f"corretti: %s/%s" % (str(total_corrects),str(len(test_dl))))
        
    print(f"Epoca: %s --> Loss: %s" % (str(epoch),str(np.round(running_loss,3))))

print('Finished Training')

corretti: 300/81
corretti: 333/81
corretti: 333/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 333/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/81
corretti: 300/

KeyboardInterrupt: ignored